In [1]:
%matplotlib inline

import sys
sys.path.append("..")
import splitwavepy as sw

import scipy
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def get_noise(y):
    """
    Return a randomly simulated noise trace with similar spectral properties to y.
    """  
    # white noise
    x = np.random.normal(y.size)
    # convolve with y
    x = np.convolve(y,x,'same')
    # additional randomisation
    x = np.roll(x,np.random.randint(y.size))
    # whipeout near nyquist
    x = np.convolve(np.array([1,1,1]),x,'same')
    # normalise energy
    x = x * (np.sum(y**2) / np.sum(x**2))
    # return
    return x


In [ ]:
a = sw.Pair(delta=0.1,noise=0.001,fast=30,lag=1.2)

In [ ]:
a.plot()

In [ ]:
def bootstrap_sample(data,fast,lag):    
    # copy original data
    bs = data.copy()     
    # replace noise sequence
    bs.unsplit(fast,lag)
    srcpol = core.pca(bs.x,bs.y)
    bs.rotateto()
    bs.y = get_noise(bs.y)
    bs.split(fast,lag)
    return bs

In [ ]:
b = bootstrap_sample(a,30,1.2)
b.plot()

In [ ]:
plt.plot(get_noise(a.y))
# plt.plot(a.y)

In [ ]:
plt.plot(a.srcpoldata_corr())

In [ ]:
a.plot()

In [ ]:
m = sw.EigenM(a)

In [ ]:
m.srcpol

In [ ]:
a.plot()

In [ ]:
sw.core.pca(m.data_corr.x,m.data_corr.y)

In [ ]:
m.data_corr